In [8]:
from PyQt6.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QTextEdit, QHBoxLayout
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qtagg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import networkx as nx
import numpy as np
from PyQt6 import QtWidgets
import itertools


In [9]:
class NearestNeighbor:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.adj_list = {city: {} for city in range(self.num_cities)}
        self.generate_random_graph()

    def add_distance(self, city1, city2, distance):
        self.adj_list[city1][city2] = distance
        self.adj_list[city2][city1] = distance

    def generate_random_graph(self):
        for city1 in range(self.num_cities):
            for city2 in range(city1 + 1, self.num_cities):
                distance = np.random.randint(50, 100)
                self.add_distance(city1, city2, distance)

    def heuristic(self, current_city, visited):
        min_distance = float('inf')
        nearest_neighbor = None

        for neighbor, distance in self.adj_list[current_city].items():
            if not visited[neighbor] and distance < min_distance:
                min_distance = distance
                nearest_neighbor = neighbor
        return nearest_neighbor, min_distance

    def nearest_neighbor(self, start_city):
        visited = [False] * self.num_cities
        path_nns = [start_city]
        visited[start_city] = True
        visited_count = 1

        total_distance = 0

        while visited_count < self.num_cities:
            current_city = path_nns[-1]
            nearest_neighbor, min_distance = self.heuristic(current_city, visited)

            if nearest_neighbor is not None:
                path_nns.append(nearest_neighbor)
                visited[nearest_neighbor] = True
                visited_count += 1
                total_distance += min_distance
            else:
                break

        path_nns.append(start_city)
        total_distance += self.adj_list[path_nns[-2]][start_city] 

        return path_nns, total_distance

In [10]:
class Graph_CP:
    def __init__(self, vertices):
        self.vertices = vertices

    def get_neighbors(self, v, edges):
        neighbors = []
        for edge in edges:
            if v not in edge:
                continue
            neighbors.append(edge[0] if edge[0] != v else edge[1])
        return neighbors

    def get_all_edges(self):
        edge_list = []
        for edge in itertools.combinations(self.vertices.keys(), 2):
            edge_list.append(edge)
        return edge_list

In [11]:
class Closet_Pair:
    def perform_algorithm(self, graph):
        visited = []
        all_edges = graph.get_all_edges()
        edges_sorted = sorted(all_edges, key=lambda x: graph.vertices[x[0]][x[1]])
        added_edges = []
        for edge in edges_sorted:
            degree_map = self.get_degree_map(graph, added_edges)
            if len(added_edges) == len(graph.vertices) - 1:
                degrees_sorted = sorted(list(degree_map.keys()), key=lambda x: degree_map[x])
                final_edge = (degrees_sorted[0], degrees_sorted[1])
                added_edges.append(final_edge)
                break
            u, v = edge
            if degree_map[u] == 2 or degree_map[v] == 2:
                continue
            if self.is_connected(graph, u, v, added_edges):
                continue
            added_edges.append(edge)
        return added_edges

    def get_degree_map(self, graph, edges):
        v_to_degree = {v: 0 for v in graph.vertices}
        for edge in edges:
            u, v = edge
            v_to_degree[u] = v_to_degree.get(u, 0) + 1
            v_to_degree[v] = v_to_degree.get(v, 0) + 1
        return v_to_degree

    def is_connected(self, graph, u, v, edges):
        visited = set()
        def dfs(current):
            visited.add(current)
            for neighbor in graph.get_neighbors(current, edges):
                if neighbor not in visited:
                    dfs(neighbor)
        dfs(u)
        return v in visited

    def calculate_total_cost(self, graph, edges):
        total_cost_gm = 0
        for edge in edges:
            u, v = edge
            total_cost_gm += graph.vertices[u][v]
        return total_cost_gm

In [12]:
class Plotter:
    def __init__(self, positions, path, start_city, title):
        self.positions = positions
        self.path = path
        self.start_city = start_city
        self.title = title

    def plot(self, canvas, subplot_position):
        ax = canvas.figure.add_subplot(subplot_position)
        ax.set_title(self.title)
        for node, pos in self.positions.items():
            ax.scatter(pos[0], pos[1], c='red' if node == self.start_city else 'blue')
            offset = 0.035  
            ax.text(pos[0] - offset, pos[1] + offset, str(node), fontsize=12, ha='right', va='bottom')

        for i in range(len(self.path) - 1):
            start_pos = self.positions[self.path[i]]
            end_pos = self.positions[self.path[i + 1]]
            ax.annotate("",
                        xy=end_pos, xycoords='data',
                        xytext=start_pos, textcoords='data',
                        arrowprops=dict(arrowstyle="->",
                                        connectionstyle="arc3"))

        ax.axis('off')

In [13]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Greedy for TSP")
        self.setGeometry(100, 100, 1200, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout(self.central_widget)

        self.canvas = FigureCanvas(Figure(figsize=(10, 5)))

        layout.addWidget(self.canvas)

        controls_layout = QHBoxLayout()
        layout.addLayout(controls_layout)

        self.num_cities_spinbox = QtWidgets.QSpinBox()
        self.num_cities_spinbox.setStyleSheet("background-color: #FFFDD0;")

        self.num_cities_spinbox.setMinimum(2)
        controls_layout.addWidget(QLabel("Number of cities:"))
        controls_layout.addWidget(self.num_cities_spinbox)

        self.start_button = QtWidgets.QPushButton("Bắt đầu")
        self.start_button.setStyleSheet("background-color: #FFFFCC;") 
        self.start_button.clicked.connect(self.start_algorithm)
        controls_layout.addWidget(self.start_button)

        self.status_bar = self.statusBar()
        self.status_bar.showMessage("Nhập số lượng thành phố và bấm Bắt đầu")

        self.path_text_edit_nns = QTextEdit()
        self.path_text_edit_nns.setReadOnly(True)

        self.path_text_edit_gm = QTextEdit()
        self.path_text_edit_gm.setReadOnly(True)

        paths_layout = QHBoxLayout()
        layout.addLayout(paths_layout)

        paths_layout.addWidget(QLabel("Quy trình đi từng bước (NNS):"))
        paths_layout.addWidget(self.path_text_edit_nns)
        self.path_text_edit_nns.setStyleSheet("background-color: #f2c867;") 

        paths_layout.addWidget(QLabel("Quy trình đi từng bước (GM):"))
        paths_layout.addWidget(self.path_text_edit_gm)
        self.path_text_edit_gm.setStyleSheet("background-color: #f2c867;")  

        self.distance_label = QLabel("Tổng quãng đường: 0")
        layout.addWidget(self.distance_label)

        self.graph = None
        self.positions = None
        self.path_nns = None
        self.start_city = None
        self.total_distance_nns = 0
        self.total_distance_gm = 0

    def start_algorithm(self):
        num_cities = self.num_cities_spinbox.value()
        NNS = NearestNeighbor(num_cities)
        self.start_city = np.random.randint(0, NNS.num_cities - 1)
        self.path_nns, self.total_distance_nns = NNS.nearest_neighbor(self.start_city)

        graph_data = NNS.adj_list
        graph = Graph_CP(graph_data)
        closet_pair = Closet_Pair()
        self.edges_gm = closet_pair.perform_algorithm(graph)
        self.total_distance_gm = closet_pair.calculate_total_cost(graph, self.edges_gm)
        self.path_gm = self.get_path_from_edges(self.edges_gm, self.start_city)

        G = nx.Graph()
        for city, neighbors in NNS.adj_list.items():
            for neighbor, distance in neighbors.items():
                G.add_edge(city, neighbor, weight=distance)
        self.positions = nx.spring_layout(G, weight='weight')

        self.plot_result()

        self.path_text_edit_nns.clear()
        self.path_text_edit_nns.append("Đồ thị:")
        for city, neighbors in NNS.adj_list.items():
            self.path_text_edit_nns.append(f"{city}: {neighbors}")
        self.path_text_edit_nns.append("\nQuy trình đi từng bước (NNS):")
        for i, city in enumerate(self.path_nns[:-1]):
            self.path_text_edit_nns.append(f"Bước {i + 1}: Đến thành phố {city}")
        self.path_text_edit_nns.append(f"Bước {len(self.path_nns)}: Quay lại thành phố {self.path_nns[-1]}")

        self.path_text_edit_gm.clear()
        self.path_text_edit_gm.append("\nQuy trình đi từng bước (GM):")
        for i, edge in enumerate(self.edges_gm):
            self.path_text_edit_gm.append(f"Bước {i + 1}: Thêm cạnh {edge}")
        self.distance_label.setText(f"Tổng quãng đường (NNS): {self.total_distance_nns}, Tổng quãng đường (Closet Pair): {self.total_distance_gm}")

    def get_path_from_edges(self, edges, start_city):
        edge_dict = {u: v for u, v in edges}
        path = [start_city]
        current_city = start_city
        while len(path) < len(edges) + 1:
            next_city = edge_dict.get(current_city, None)
            if next_city is None:
                break
            path.append(next_city)
            current_city = next_city
        path.append(start_city)
        return path

    def plot_result(self):
        self.canvas.figure.clf()

        plotter_nns = Plotter(self.positions, self.path_nns, self.start_city, "Nearest Neighbor Solution")
        plotter_nns.plot(self.canvas, 131)

        ax_gm = self.canvas.figure.add_subplot(133)
        ax_gm.set_title("Greedy Method Solution")

        for node, pos in self.positions.items():
            ax_gm.scatter(pos[0], pos[1], c='blue')
            offset_gm = 0.035 
            ax_gm.text(pos[0] - offset_gm, pos[1] + offset_gm, str(node), fontsize=12, ha='right', va='bottom')

        ax_gm.scatter(self.positions[self.start_city][0], self.positions[self.start_city][1], c='red')

    
        for edge in self.edges_gm:
            u, v = edge
            ax_gm.plot([self.positions[u][0], self.positions[v][0]], [self.positions[u][1], self.positions[v][1]], c='black')

        ax_gm.axis('off')

        self.canvas.draw()

In [14]:
if __name__ == "__main__":
    import sys
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3056\1855424514.py:3: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  super().__init__()


SystemExit: 0

C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
